이번 스텝에서는 8번 노드에 나왔던 모델과 동일한 모델을 실습합니다. 하지만 다양한 성능 향상 방법을 사용하여 8번 노드에서의 딥러닝 모델보다 성능을 더 높일 수 있습니다. 8번 노드에서 사용했던 ReLU와는 다른 활성화함수를 사용해보고, BatchNormalization과 Dropout을 적용해 봅시다. ModelCheckpoint와 EarlyStopping 콜백 함수도 적용해 보세요.

#데이터 로드 및 전처리
reuters 데이터셋을 로드하고, 훈련 데이터셋에서 25%를 검증 데이터셋으로 분리합니다. 데이터셋은 keras.datasets에서 불러올 수 있는 reuters 데이터 셋을 활용하고, 이때 파라메터 조정을 통해 단어의 개수는 10000개로 제한합니다(num_words=10000).
데이터셋 분리 작업에서는 sklearn 라이브러리의 train_test_split 함수를 활용합니다.
훈련, 검증, 테스트 데이터와 레이블 종류가 몇개인지 출력합니다.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from tensorflow.keras import models, layers
from tensorflow.keras import callbacks
from tensorflow.keras import utils
from keras.datasets import reuters

In [6]:
# 문제 2-1: reuters 데이터셋을 로드하고, 훈련 데이터셋에서 25%를 검증 데이터셋으로 분리합니다.
from tensorflow.keras.datasets import reuters
from sklearn.model_selection import train_test_split

(train_data, train_labels), (test_data, test_labels) = reuters.load_data(num_words = 10000)

# 검증데이터 셋으로 25%분리
# x_train, x_val, y_train, y_val = train_test_split(train_data, train_labels, test_size=0.25, random_state=42)
x_train_raw, x_val_raw, y_train_raw, y_val_raw = train_test_split(train_data, train_labels, test_size=0.25, random_state=42)


# 훈련, 검증, 테스트 데이터와 레이블 종류가 몇 개인지 출력합니다.
print(len(train_data))
print(len(x_train))
print(len(x_val))
print(len(test_data))
print(len(set(train_labels)))

8982
8982
2246
2246
46


텍스트 데이터의 전처리를 위해서 각 데이터셋마다 원-핫 인코딩을 수행합니다. 원-핫 인코딩 함수를 직접 만들어보세요.
레이블 데이터들은 범주형 형태로 to_categorical() 함수를 사용해 변환합니다.

In [7]:
# 문제 2-2: 텍스트 데이터의 전처리를 위해서 각 데이터셋마다 원-핫 인코딩을 수행합니다.
from tensorflow.keras.utils import to_categorical

# Text one-hot coding
def one_hot_encoding(data, dim=10000):
  results = np.zeros((len(data), dim))
  for i, sequence in enumerate(data):
      results[i, sequence] = 1.0

  return results

# 각 데이터셋 one-hot coding
# x_train = one_hot_encoding(train_data)
# x_val = one_hot_encoding(x_val)
# x_test = one_hot_encoding(test_data)
x_train = one_hot_encoding(x_train_raw)
x_val = one_hot_encoding(x_val_raw)
x_test = one_hot_encoding(test_data)

# 레이블 데이터들은 범주형 형태로 to_categorical() 함수를 사용해 변환합니다.
# y_train = to_categorical(y_train)
# y_val = to_categorical(y_val)
# y_test = to_categorical(test_labels)
y_train = to_categorical(y_train_raw)
y_val = to_categorical(y_val_raw)
y_test = to_categorical(test_labels)


#모델 구성 및 컴파일
BatchNormalization과 Dropout을 적용하여 빠르게 학습하면서 과대적합을 방지하고, 46개의 뉴스 주제를 분류하는 딥러닝 모델을 구성합니다.

In [8]:
# 문제 2-3: 빠른 학습과 과대적합을 방지하기 위해 BatchNormalization과 Dropout을 적용한 딥러닝 모델을 구성합니다.
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization

model = Sequential([
    Dense(512, activation = 'relu', input_shape=(10000,), name = 'input'),
    BatchNormalization(),
    Dropout(0.5),

    Dense(256, activation ='relu', name='hidden1'),
    BatchNormalization(),
    Dropout(0.5),

    Dense(46, activation = 'softmax', name='output')
])



In [9]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input (Dense)                   │ (None, 512)            │     5,120,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 512)            │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ hidden1 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ (None, 46)             │        11,822 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,266,734 (20.09 MB)

 Trainable params: 5,265,198 (20.09 MB)

 Non-trainable params: 1,536 (6.00 KB)

#모델 학습
ModelCheckpoint와 EarlyStopping 콜백 함수를 적용하여 모델 학습을 진행합니다.

In [10]:
# 문제 2-4: ModelCheckpoint와 EarlyStopping 콜백 함수를 적용하여 모델 학습을 진행합니다.
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# 모델 컴파일
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

checkpoint_cb = ModelCheckpoint('best_model.h5', save_best_only=True, monitor='val_loss', mode='min')
earlystop_cb = EarlyStopping(patience = 10, restore_best_weights=True, monitor='val_loss', mode='min')

# 모델 학습
history = model.fit(x_train, y_train,
                    epochs = 100,
                    batch_size = 32,
                    validation_data = (x_val, y_val),
                    callbacks= [checkpoint_cb, earlystop_cb])

Epoch 1/100
211/211 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - accuracy: 0.5156 - loss: 2.4814

211/211 ━━━━━━━━━━━━━━━━━━━━ 18s 75ms/step - accuracy: 0.5162 - loss: 2.4784 - val_accuracy: 0.7640 - val_loss: 1.3827
Epoch 2/100
211/211 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - accuracy: 0.8091 - loss: 0.8730

211/211 ━━━━━━━━━━━━━━━━━━━━ 18s 84ms/step - accuracy: 0.8091 - loss: 0.8728 - val_accuracy: 0.8126 - val_loss: 0.8943
Epoch 3/100
211/211 ━━━━━━━━━━━━━━━━━━━━ 17s 82ms/step - accuracy: 0.8826 - loss: 0.5110 - val_accuracy: 0.8232 - val_loss: 0.9489
Epoch 4/100
211/211 ━━━━━━━━━━━━━━━━━━━━ 15s 72ms/step - accuracy: 0.9218 - loss: 0.3265 - val_accuracy: 0.8232 - val_loss: 1.0322
Epoch 5/100
211/211 ━━━━━━━━━━━━━━━━━━━━ 18s 86ms/step - accuracy: 0.9328 - loss: 0.2761 - val_accuracy: 0.8197 - val_loss: 0.9959
Epoch 6/100
211/211 ━━━━━━━━━━━━━━━━━━━━ 19s 81ms/step - accuracy: 0.9420 - loss: 0.2276 - val_accuracy: 0.8192 - val_loss: 1.0479
Epoch 7/100
211/211 ━━━━━━━━━━━━━━━━━━━━ 16s 78ms/step - accuracy: 0.9419 - loss: 0.2222 - val_accuracy: 0.8232 - val_loss: 1.0593
Epoch 8/100
211/211 ━━━━━━━━━━━━━━━━━━━━ 16s 77ms/step - accuracy: 0.9498 - loss: 0.2113 - val_accuracy: 0.8175 - val_loss: 1.0670
Epoch 9/100
211/211 ━━━━━━━━━━━━━━━━━━━━ 16s 77ms/step - accuracy: 0.9558 - loss: 0.1927 - val_

학습 히스토리의 loss, val_loss, accuracy, val_accuracy를 차트로 시각화합니다.

In [14]:
# 학습 히스토리의 `loss`, `val_loss`, `accuracy`, `val_accuracy`를 차트로 시각화합니다.
plt.style.use('seaborn-v0_8-white')

history_dict = history.history

loss = history_dict['loss']
val_loss = history_dict['val_loss']

epochs = range(1, len(loss) + 1)
fig = plt.figure(figsize=(12, 5))

ax1 = fig.add_subplot(1, 2, 1)
ax1.plot(epochs, loss, 'b-', label='train_loss')
ax1.plot(epochs, val_loss, 'r-', label='val_loss')
ax1.set_title('Train and Validation Loss')
ax1.set_xlabel('Epochs')
ax1.set_ylabel('Loss')
ax1.grid()
ax1.legend()

accuracy = history_dict['accuracy']
val_accuracy = history_dict['val_accuracy']

ax2 = fig.add_subplot(1, 2, 2)
ax2.plot(epochs, accuracy, 'b-', label='train_accuracy')
ax2.plot(epochs, val_accuracy, 'r-', label='val_accuracy')
ax2.set_title('Train and Validation Accuracy')
ax2.set_xlabel('Epochs')
ax2.set_ylabel('Accuracy')
ax2.grid()
ax2.legend()

plt.show()

# 모델 평가 및 예측
테스트 데이터셋을 이용해 모델을 평가합니다.

In [15]:
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=1)

print(f"테스트 손실 (Loss): {test_loss:.4f}")
print(f"테스트 정확도 (Accuracy): {test_acc:.4f}")


71/71 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.7937 - loss: 0.9498
테스트 손실 (Loss): 0.9627
테스트 정확도 (Accuracy): 0.7912
